

**Spark Driver**: The driver is the program space in which Spark Application's main method runs coodinating the Spark entire application.

## Executors

- An executor is a processing engine running on each worker node.

3. If there are more slots than tasks the spark job will likely not run as efficiently as possible.
4. The most granular unit of work in Spark is a task.
5. What is correct about Spark Jobs?
- Jobs are broken down into stages.
- There are multiple tasks within a single job when a DataFrame has more than one partition.
- Jobs are collections of tasks that are divided up based on when an action is called.
- Jobs are collections of tasks that are divided based on physical plan optimizations.

6. Operations to most likely to result in a Shuffle:

- Join
- GroupBy

7. The default value of `spark.sql.shuffle.partitions` is 200. Therefore, by default dataframes will be split into 200 unique partitions when data is being shuffled.
8. A process is lazily evaluated if its execution does not start until it is put into action by some type of trigger.

9. Operations classified as an action:
- take(n): returns an array with the first n elements of the dataset.
- collect(): returns all the elements of the dataset as an array at the driver program.
- count(): returns the number of elements in the dataset.
- show():

10. Dataframe operations classified as wide transformations:

- groupBy()
- join()
- sort()

11. The cluster execution mode runs the driver on a worker node within a cluster, while the client execution mode runs the driver on the client machine (also known as a gateway or edge node).

12. Spark Stability capabilities:

- Spark is designed to support the loss of any set of worker nodes, using lineage information to recover lost data.
- Spark will rerun any failed tasks due to failed worker nodes.
- Spark will spill data to disk if it does not fit in memory.
- Spark will recompute data cached on failed worker nodes.

13. When is most advantageous to store DataFrame df at the MEMORY_AND_DISK storage level rather than the MEMORY_ONLY storage level?

- When it is faster to read all the computed data in Dafraframe DF that cannot fit into memory from disk rather than recompute it based on its logical plan.

14. Which configuration of cluster is most likely to experience a out-of-memory error in response to data skew in a single partition?

- Clusters with more worker nodes.

15. If I have 2 dataframes, df_a with 128GB and df_b with 1GB, it's most recommended to broadcast df_b to all worker nodes to avoid the shuffle of df_a.

16. If I have a dataframe with 8 partitions and I would like to repartition it to 12 partitions I most use `df.repartition(12)`.

17. The cache() operation can only cache a DataFrame at the MEMORY_AND_DISK level (the default). persist() should be used to cache a DataFrame at the MEMORY_ONLY level.

18. 

- `spark.sql.adaptative.coalescePartitions.enabled`: 
- `spark.sql.adaptative.skewJoin.enabled`:
- `spark.sql.autoBroadcastJoinThreshold`:
- `spark.sql.adaptive.skeewedJoin.skewThreshold`:

19. Understand the difference between types Timestamp and Date in Spark:

- A Timestamp type is a date and time with a precision of milliseconds.

20. Storage Levels

- MEMORY_ONLY_2
- MEMORY_AND_DISK_SER
- MEMORY_AND_DISK
- MEMORY_ONLY_SER_2
- MEMORY_ONLY




In [14]:
import pyspark
from pyspark.sql import SparkSession
import os


def get_spark_session():
  NESSIE_URI = "http://nessie:19120/api/v1"
  MINIO_HOST = 'http://minio:9000'
  MINIO_ACCESS_KEY = "pTo0VEtqzA7hTKsCwCOV"
  MINIO_SECRET_KEY = "B7j2Pt93vVDkYhJ9BsKBi43ZhQtMZU5iP4UznW92"

  jar_packages = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1",
    "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.95.0",
    "software.amazon.awssdk:bundle:2.17.178",
    "software.amazon.awssdk:url-connection-client:2.17.178"]

  spark_extensions = [
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"]

  conf = (
    pyspark.SparkConf()
      .setAppName('iceberg_hello_world')
      .set('spark.jars.packages', ','.join(jar_packages))
      .set('spark.sql.extensions', ','.join(spark_extensions))
      .set('spark.sql.catalog.nessie', "org.apache.iceberg.spark.SparkCatalog")
      .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
      .set('spark.sql.catalog.nessie.ref', 'main')
      .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
      .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
      .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
      .set('spark.sql.catalog.nessie.s3.endpoint', MINIO_HOST)
      .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
      .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
      .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
      .set('spark.hadoop.fs.s3a.path.style.access', 'true')
      .set('spark.hadoop.fs.s3a.connection.timeout', '600000')
      .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
      .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
      .set('spark.hadoop.fs.s3a.endpoint', MINIO_HOST))
  return SparkSession.builder.config(conf=conf).master("spark://spark-master:7077").getOrCreate()



spark = get_spark_session()
spark.sql("SHOW TABLES IN nessie").show()
spark.sql("SHOW DATABASES").show()

# df.select("col_1", "col_2").show()
# df.drop("col_1", "col_2").show()
# df.filter(col("sqft") <= 25000).show()
# df.filter((col("sqft") <= 25000) | (col("customerSatisfaction") >= 30)).show()
# df.withColumn("storedId", col("storeId").cast("string")).show()
# df.withColumn("modality", lit("PHYSICAL"))
# df.withColumn("store_value_category", col("store_category".split("_")[0])).withColumn("store_size_category", col("store_category").split("_")[0])).show()
# df.withColumn("productCategories", explode(col("productCategories"))).show()
# df.withColumn("storeDescription", regexp_replace(col("storeDescription"), "^Description", "")).show()
# df.distinct().show()
# df.drop
# df.dropDuplicates().show()
# df.drop_duplicates().show()
# df.dropDuplicates(subset="all").show()
# df.agg(approx_count_distinct(col("division"))).alias("count").show()
# df.describe().show()
# df.summary(*statistics).show()

# df.orderBy("division", ascending=False).show()
# df.sort("division", ascending=False).show()
# df.sort(desc("division")).show()
# df.sort(col("division").desc()).show()


# df.sample(withReplacement=False, fraction=0.5, seed=42).show()
# spark.udf.register("ASSESS_PERFORMANCE", assessPerformance)
# df.withColumn("performance", expr("ASSESS_PERFORMANCE(sales, target)")).show()
# assessPerformance = udf(assessPerformance)
# df.withColumn("performance", assessPerformance(col("sales")).show()
# df.withColumn("splited_col", split(col("col"), " ").getItem(0)).show()


# df.sample(True, fraction=0.1, seed=42).show()
# df.na.fill(30000, col("sqft")).show()
# df.na.fill(30000, ["sqft"]).show()
# df.agg



from pyspark.sql.types import IntegerType

df = spark.createDataFrame([1,2,3], IntegerType())

df.show()




+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |open_2023|      false|
+---------+---------+-----------+

+---------+
|namespace|
+---------+
|  default|
+---------+



+-----+
|value|
+-----+
|    1|
|    2|
|    3|
+-----+

